In [1]:
import sys

sys.path.append("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning")
import os
os.chdir("/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning")

import NLP_utils.preprocessing as nlp_processing
from RL_for_NLP.text_environments import TextEnvClfWithBertTokens, TextEnvClf, TextEnvClfForBertModels
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm

from RL_for_NLP.text_data_pools import PartialReadingDataPoolWithTokens, PartialReadingDataPoolWithBertTokens
import reinforce_algorithm_utils as rl_monte_carlo
import policy_networks as pn

import torch as th
from torch.optim import Adam
import mlflow
from RL_for_NLP.text_reward_functions import calculate_stats_from_cm

import actor_critic_algortihm_utils as a2c_utils 

from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3 import DQN

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import json
from collections import Counter
from tqdm import tqdm
from tqdm.notebook import tqdm

Using TensorFlow backend.


In [4]:
x = 3.123456
xx = x :.4
xx

SyntaxError: invalid syntax (617518852.py, line 2)

In [2]:
data = nlp_processing.openDfFromPickle("NLP_datasets/ag_news/ag_news_train_distilbert-base-uncased.pkl")
pool = PartialReadingDataPoolWithBertTokens(data, "text", "label", 512, 50, mask = True)
env = TextEnvClfForBertModels(pool, 30522, int(1e+5), "score", True)

Maximum sentence length in pool: 512


Padding the data and populating samples...: 100%|██████████| 96000/96000 [00:08<00:00, 11105.59it/s]

---- Inside constructor ---------
Input id shape/type:  (50,) <class 'numpy.ndarray'>
Attn mask shape/type:  (50,) <class 'numpy.ndarray'>
------------- End Constructor-------------


In [3]:
policy = a2c_utils.DistibertActorCriticPolicy(50, 4)

In [4]:
policy(th.tensor(env.current_state["input_id"]).unsqueeze(0), th.tensor(env.current_state["attn_mask"]).unsqueeze(0))

(tensor([[0.2551, 0.2360, 0.2752, 0.2338]], grad_fn=<SoftmaxBackward>),
 tensor([[0.0738]], grad_fn=<AddmmBackward>))

In [5]:
optimizer = Adam(policy.parameters())
a2c = a2c_utils.ActorCriticAlgorithmBertModel(policy, env, optimizer, gamma=1.)

In [6]:
a2c.train_a2c(100, 100, log_interval=10)

Average reward so far: 0.000 (updated every 10 episodes):   0%|          | 0/100 [00:00<?, ?it/s]/Users/emrebaloglu/Documents/RL/basic_reinforcement_learning/actor_critic_algortihm_utils.py:282: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  value_losses.append(F.smooth_l1_loss(value, th.tensor([R])))
Average reward so far: 0.000 (updated every 10 episodes):   2%|▏         | 2/100 [01:07<55:15, 33.83s/it]


KeyboardInterrupt: 

In [27]:
from transformers import DistilBertModel, DistilBertTokenizer, BertModel, BertTokenizer
import torch as th

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", max_length=15, padding="max_length")
distilbert = BertModel.from_pretrained("bert-base-uncased")

In [30]:
out = distilbert(**inputs, return_dict=False)
print(len(out))

2


In [36]:
out[0].shape

torch.Size([1, 15, 768])

In [35]:
out[1].shape

torch.Size([1, 768])

In [42]:
cnn = th.nn.Conv1d(15, 1, kernel_size = 16)
th.nn.Flatten(cnn(out[0]).shape)


Flatten(start_dim=torch.Size([1, 1, 753]), end_dim=-1)

In [26]:
distilbert.add_module(th.nn.Flatten())
distilbert.add_module(th.nn.Linear(50*768, 64))


TypeError: add_module() missing 1 required positional argument: 'module'

In [69]:
import gym
from gym import spaces
import numpy as np
import cv2
import random
import time
from collections import deque

SNAKE_LEN_GOAL = 30


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if snake_head[0] >= 500 or snake_head[0] < 0 or snake_head[1] >= 500 or snake_head[1] < 0:
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.prev_actions = deque(maxlen=SNAKE_LEN_GOAL)  # however long we aspire the snake to be
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=-500, high=500,
                                            shape=(5 + SNAKE_LEN_GOAL,), dtype=int)

    def step(self, action):
        self.prev_actions.append(action)
        cv2.imshow('a', self.img)
        cv2.waitKey(1)
        self.img = np.zeros((500, 500, 3), dtype='uint8')
        # Display Apple
        cv2.rectangle(self.img, (self.apple_position[0], self.apple_position[1]),
                      (self.apple_position[0] + 10, self.apple_position[1] + 10), (0, 0, 255), 3)
        # Display Snake
        for position in self.snake_position:
            cv2.rectangle(self.img, (position[0], position[1]), (position[0] + 10, position[1] + 10), (0, 255, 0), 3)

        # Takes step after fixed time
        t_end = time.time() + 0.05
        k = -1
        while time.time() < t_end:
            if k == -1:
                k = cv2.waitKey(1)
            else:
                continue

        button_direction = action
        # Change the head position based on the button direction
        if button_direction == 1:
            self.snake_head[0] += 10
        elif button_direction == 0:
            self.snake_head[0] -= 10
        elif button_direction == 2:
            self.snake_head[1] += 10
        elif button_direction == 3:
            self.snake_head[1] -= 10

        # Increase Snake length on eating apple
        if self.snake_head == self.apple_position:
            self.apple_position, self.score = collision_with_apple(self.apple_position, self.score)
            self.snake_position.insert(0, list(self.snake_head))

        else:
            self.snake_position.insert(0, list(self.snake_head))
            self.snake_position.pop()

        # On collision kill the snake and print the score
        if collision_with_boundaries(self.snake_head) == 1 or collision_with_self(self.snake_position) == 1:
            font = cv2.FONT_HERSHEY_SIMPLEX
            self.img = np.zeros((500, 500, 3), dtype='uint8')
            cv2.putText(self.img, 'Your Score is {}'.format(self.score), (140, 250), font, 1, (255, 255, 255), 2,
                        cv2.LINE_AA)
            cv2.imshow('a', self.img)
            self.done = True

        #self.total_reward = len(self.snake_position) - 3  # default length is 3
        #self.reward = self.total_reward - self.prev_reward
        #self.prev_reward = self.total_reward

        if self.done:
            self.reward = -10
        else:
            self.reward = self.score

        head_x = self.snake_head[0]
        head_y = self.snake_head[1]

        apple_delta_x = self.apple_position[0] - head_x
        apple_delta_y = self.apple_position[1] - head_y

        snake_length = len(self.snake_position)

        self.prev_actions = deque(maxlen=SNAKE_LEN_GOAL)
        for _ in range(SNAKE_LEN_GOAL):
            self.prev_actions(-1)

        # create observation:
        observation = [head_x, head_y, apple_delta_x, apple_delta_y, snake_length] + list(self.prev_actions)
        observation = np.array(observation)

        info = {}
        return observation, self.reward, self.done, info

    def reset(self):
        self.img = np.zeros((500, 500, 3), dtype='uint8')
        # Initial Snake and Apple position
        self.snake_position = [[250, 250], [240, 250], [230, 250]]
        self.apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
        self.score = 0
        self.prev_button_direction = 1
        self.button_direction = 1
        self.snake_head = [250, 250]

        self.prev_reward = 0

        self.done = False

        head_x = self.snake_head[0]
        head_y = self.snake_head[1]

        apple_delta_x = self.apple_position[0] - head_x
        apple_delta_y = self.apple_position[1] - head_y
        snake_length = len(self.snake_position)

        for i in range(SNAKE_LEN_GOAL):
            self.prev_actions.append(-1)  # to create history

        # create observation:
        observation = [head_x, head_y, apple_delta_x, apple_delta_y, snake_length] + list(self.prev_actions)
        observation = np.array(observation)

        return observation


In [70]:
env = SnekEnv()
print(env.reset().shape)
print(env.observation_space)

(35,)
Box([-500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500
 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500 -500
 -500 -500 -500 -500 -500 -500 -500], [500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500
 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500], (35,), int64)


In [71]:
from stable_baselines3.common.env_checker import check_env

check_env(env)

TypeError: 'collections.deque' object is not callable

: 